# Re testing eisenscript_to_xml parser

Looking at issue https://github.com/elfnor/generative-art-examples/issues/2

```
In es_call() change:
CHANGE: atr_value = pcall.rule_name[0]
TO: atr_value = pcall[0]

Where did the 'rule_name' property come from?
```


In [2]:

import os
import glob
from pathlib import Path
from eisenscript_to_xml import eisenscript_to_xml as ex

In [3]:
fn = '../tests/test_es/koch_snowflake.es'

In [4]:
Path(fn).exists()

True

In [5]:
# with change suggested in issue
ex.translate_eisen(fn)

In [6]:
xml_dir = os.path.join(os.path.curdir, "xml_translate")
es_dir = "../tests/test_es/Examples/"
Path(es_dir).exists()

True

In [7]:
for name in glob.glob(es_dir + "*" + ".es"):
    print(name)
    try:
        ex.translate_eisen(name, xml_dir)
    except:
        print("failed")

../tests/test_es/Examples/Default.es
../tests/test_es/Examples/Moduli Creatures.es
../tests/test_es/Examples/Thingy.es
failed
../tests/test_es/Examples/SpiralTree2D.es
../tests/test_es/Examples/Nouveau.es
failed
../tests/test_es/Examples/Ball.es
../tests/test_es/Examples/Mondrian.es
../tests/test_es/Examples/Trees 3d.es
../tests/test_es/Examples/Menger.es
../tests/test_es/Examples/Pure Structure.es
failed
../tests/test_es/Examples/BinaryKite.es
../tests/test_es/Examples/Konstrukt.es
failed
../tests/test_es/Examples/Grinder.es
../tests/test_es/Examples/Torus3.es
../tests/test_es/Examples/Nouveau3.es
failed
../tests/test_es/Examples/RoundTree.es
../tests/test_es/Examples/NablaSystem.es
../tests/test_es/Examples/Synctor.es
../tests/test_es/Examples/Nabla.es
../tests/test_es/Examples/City of Glass.es
../tests/test_es/Examples/Arc Sphere.es
../tests/test_es/Examples/Torus2.es
../tests/test_es/Examples/NineWorthies.es
../tests/test_es/Examples/Octopod II.es
../tests/test_es/Examples/MeshTest

Code runs but output is malformed  
`{y 0.866 } R1`  
should translate to  
`<call transforms="ty 0.866" rule="R1"/>`  
but now gives  
`<call transforms="" rule="[['y', '0.866']]"/>`  

Which doesn't work in Sverchok, and is probably a problem somewhere else in my code not with the suggested fix.  

`/media/elfnor/BeastHoard/BlenderProjects/GenerativeArtNode/es_test-08.ipynb` translated a test of a spiral when it was last run in May 2016. That translated xml still works in current Sverchok/generative art.

In [8]:
spiral = """
set md 50

boty
 

rule boty 
{ 
    { hue 100.0 } br
     { rx 180 x 20  rz 2 } br
} 
 
rule br
{
    { rz 60 } br3
     { rz 59 } br3
    { rz 16 s 0.93 x 2 } br
}
 
rule br3
{
   { x 1 y 1 s 1 } sphere
    { x 2 y 2  } box
    { rz 1.6 x 1 y 1 s 0.980 } br3
}"""

In [9]:
# calculate using module
file_def =ex.eisen_grammar()
result = file_def.parseString(spiral)
xml = ex.es_xml2(result)
print('-----------')
print(ex.prettify(xml))

-----------
<?xml version="1.0" ?>
<rules max_depth="50">
  <rule name="entry">
    <call transforms="" rule=""/>
  </rule>
  <rule name="boty">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br3">
    <instance transforms="" shape=""/>
    <instance transforms="" shape=""/>
    <call transforms="" rule=""/>
  </rule>
</rules>



Output from old notebook
```
<?xml version="1.0" ?>
<rules max_depth="50">
  <rule name="entry">
    <call rule="boty" transforms=""/>
  </rule>
  <rule name="boty">
    <call rule="br" transforms=""/>
    <call rule="br" transforms="rx 180 tx 20 rz 2"/>
  </rule>
  <rule name="br">
    <call rule="br3" transforms="rz 60"/>
    <call rule="br3" transforms="rz 59"/>
    <call rule="br" transforms="rz 16 sa 0.93 tx 2"/>
  </rule>
  <rule name="br3">
    <instance shape="sphere" transforms="tx 1 ty 1 sa 1"/>
    <instance shape="box" transforms="tx 2 ty 2"/>
    <call rule="br3" transforms="rz 1.6 tx 1 ty 1 sa 0.980"/>
  </rule>
</rules>
```

The codes in  
`es_xml2`,   
`es_call` (apart from replacing  `pcall.rule_name[0]` with `pcall[0]`)  
 looks similar, the grammar looks a bit different

 

In [10]:
# grammar from old notebook
from pyparsing import *
#define parser grammar
# order same as Structure Synth reference when done

fnum = Combine( Word( "+-"+nums, nums ) + 
                   Optional( "." + Optional( Word( nums ) ) ) ) 
tid = oneOf('x y z s rx ry rz')('tid')
cid = oneOf('h hue sat b brightness a alpha m')  # these are color transforms that we're ignoring

tvalues = OneOrMore(fnum)('tvalues')

gtrans = Group(tid + tvalues).setResultsName('trans', listAllMatches=True)
ctrans = cid + tvalues
c2trans = Word('color') + Word(alphanums + '#')
c3trans = Word('blend') + Word(alphanums + '#') + fnum
trans = gtrans | ctrans | c2trans | c3trans

shape_words = ['box', 'grid', 'sphere', 'line']

rule_name = NotAny(Keyword('rule')) + Word(alphas, alphanums+'_') 

loop_multiplier = fnum('count') + Suppress('*')
loop = Group(Optional(loop_multiplier) + Suppress('{') + ZeroOrMore(trans) + Suppress('}')).setResultsName('loop', listAllMatches=True)

md = oneOf('md maxdepth')
md_mod = md + fnum('md') + Optional('>' + rule_name('successor_rule') )

weight = oneOf('w weight')
w_mod = weight + fnum('wm') 

shape = MatchFirst(map(Keyword, shape_words))

global_md = Word('set') + Word('maxdepth') + fnum('global_md')

# all other set statements at beginning are ignored
global_set_ignore = Word('set') + restOfLine

global_set = global_md | global_set_ignore

shape_call = (Optional(loop) + shape('shape')).setResultsName('bcall', listAllMatches=True)
rule_call =  (ZeroOrMore(loop) + rule_name('rule_name')).setResultsName('rcall', listAllMatches=True)
call = shape_call | rule_call
rule = Group(Suppress(Keyword('rule')) + rule_name('name') +(Optional(md_mod) & Optional(w_mod)) + Suppress('{') + OneOrMore(call) + Suppress('}'))

entry = Group(OneOrMore(call)).setResultsName('entry_calls', listAllMatches=True)
main = Group(OneOrMore(rule)).setResultsName('rule_defs', listAllMatches=True)
file_def2 =  ZeroOrMore(global_set) + entry + main + ZeroOrMore(global_set_ignore)
file_def2.ignore(cppStyleComment)\
# more stuff to ignore
set_initial = Word('set') + Word('seed') + Word('initial')
file_def2.ignore(set_initial)

{{{[{{{W:(est) W:(adehmptx)} Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})} | {W:(est) rest of line}}]... Group:({{{[Group:({{{[Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]}) Suppress:('*')] Suppress:('{')} [{{{Group:({x | y | z | s | rx | ry | rz {Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}...}) | {hue | h | sat | brightness | b | alpha | a | m {Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}...}} | {W:(clor) W:(#0-9A-Za-z)}} | {{W:(bdeln) W:(#0-9A-Za-z)} Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}}]...} Suppress:('}')})] {'box' | 'grid' | 'sphere' | 'line'}} | {[Group:({{{[Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]}) Suppress:('*')] Suppress:('{')} [{{{Group:({x | y | z | s | rx | ry | rz {Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}...}) | {hue | h | sat | brightness | b | alpha | a | m {Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}...}} | {W:(clor) W:(#0-9A-Za-z)}} | {{W:(bdeln) W:(#0-9A-Za-z)} Combine:({W:(+-0-9, 0-9) ['.' [W:(0-9)]]})}}]...} Suppress:('}')})]... {~{'rule'} W:(A-Za-z, 0-9A-Z_a-z)

In [11]:
result2 = file_def2.parseString(spiral)
xml = ex.es_xml2(result2)
print('-----------')
print(ex.prettify(xml))

-----------
<?xml version="1.0" ?>
<rules max_depth="50">
  <rule name="entry">
    <call transforms="" rule=""/>
  </rule>
  <rule name="boty">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br3">
    <instance transforms="" shape=""/>
    <instance transforms="" shape=""/>
    <call transforms="" rule=""/>
  </rule>
</rules>



Which is simarly malformed. Suspect changes in pyparsing.  
The problem is in the lines for transforms and instances  
`{ x 1 y 1 s 1 } sphere`  
should be
`<instance shape="sphere" transforms="tx 1 ty 1 sa 1"/>`  
but is 
`<instance transforms="" shape=""/>`  
This is defined by `shape_call` in grammar

```python
    shape_call = (pp.Optional(loop) + shape("shape")).setResultsName(
        "bcall", listAllMatches=True
    )
```

So I'm guessing the problem  is with the definition of `loop`

Clue 

`rule_name` is defined in grammar

```
    rule_name = pp.NotAny(pp.CaselessKeyword("rule")) + pp.Word(
        pp.alphas, pp.alphanums + "_"
    )
```

Is the problem just that you can no longer access it via `pcall.rule_name` ??  




In [12]:
for p in result.entry_calls:
    print(p.dump())
    print('------------------')

['boty']
- rcall: [['boty']]
  [0]:
    ['boty']
- rule_name: ['boty']
------------------


This matches old notebook.
So problem is in `es_xml2(result)` or more likely `es_call`  

So how do you do `atr_value = pcall.rule_name[0]` in latest pyparsing?

maybe drop the index?


In [13]:
for p in result.rule_defs[0]:
    print(p.dump())
    print('------------------')

['boty', ['hue', '100.0'], 'br', [['rx', '180'], ['x', '20'], ['rz', '2']], 'br']
- loop: [['rx', '180'], ['x', '20'], ['rz', '2']]
  - trans: [['rx', '180'], ['x', '20'], ['rz', '2']]
    [0]:
      ['rx', '180']
      - tid: 'rx'
      - tvalues: ['180']
    [1]:
      ['x', '20']
      - tid: 'x'
      - tvalues: ['20']
    [2]:
      ['rz', '2']
      - tid: 'rz'
      - tvalues: ['2']
  [0]:
    ['rx', '180']
    - tid: 'rx'
    - tvalues: ['180']
  [1]:
    ['x', '20']
    - tid: 'x'
    - tvalues: ['20']
  [2]:
    ['rz', '2']
    - tid: 'rz'
    - tvalues: ['2']
- name: ['boty']
- rcall: [[['hue', '100.0'], 'br'], [[['rx', '180'], ['x', '20'], ['rz', '2']], 'br']]
  [0]:
    [['hue', '100.0'], 'br']
    [0]:
      ['hue', '100.0']
      - tvalues: ['100.0']
    [1]:
      br
  [1]:
    [[['rx', '180'], ['x', '20'], ['rz', '2']], 'br']
    [0]:
      [['rx', '180'], ['x', '20'], ['rz', '2']]
      - trans: [['rx', '180'], ['x', '20'], ['rz', '2']]
        [0]:
          ['rx', '

Comparing the output of this with original, lots of loop indices seem to have been dropeed.  
for example:  

```
- loop: 
  [0]:
    ['hue', '100.0']
    - tvalues: ['100.0']
  [1]:
    [['rx', '180'], ['x', '20'], ['rz', '2']]
```
is now

```
- loop: [['rx', '180'], ['x', '20'], ['rz', '2']]
```

the defintion of loop  
notebook
```
loop = Group(Optional(loop_multiplier) + Suppress('{') + ZeroOrMore(trans) + Suppress('}')).setResultsName('loop', listAllMatches=True)
```
module  
```
   loop_multiplier = fnum("count") + pp.Suppress("*")
    loop = pp.Group(
        pp.Optional(loop_multiplier)
        + pp.Suppress("{")
        + pp.ZeroOrMore(trans)
        + pp.Suppress("}")
    ).setResultsName("loop", listAllMatches=True)
```
are the same, so pyparsing has changed its indexing.

Look through this https://github.com/pyparsing/pyparsing/blob/master/CHANGES
 to try and find change.  

 To me, its parsing the grammar OK but the storage structure has changeed. Need to rewrite  `es_call`  and probably `es_call``es_xml2`  as well.

In May 2016 i was using version 2.1.3 or earlier
My problems seem to result from changes described in version 2.3.0 October 2018.


> - POSSIBLE API CHANGE: Fixed bug where a parse action that explicitly
  returned the input ParseResults could add another nesting level in
  the results if the current expression had a results name.

        vals = pp.OneOrMore(pp.pyparsing_common.integer)("int_values")

        def add_total(tokens):
            tokens['total'] = sum(tokens)
            return tokens  # this line can be removed

        vals.addParseAction(add_total)
        print(vals.parseString("244 23 13 2343").dump())

>  Before the fix, this code would print (note the extra nesting level):

    [244, 23, 13, 2343]
    - int_values: [244, 23, 13, 2343]
      - int_values: [244, 23, 13, 2343]
      - total: 2623
    - total: 2623

>  With the fix, this code now prints:

    [244, 23, 13, 2343]
    - int_values: [244, 23, 13, 2343]
    - total: 2623


> If you have current code which relies on this behavior, then add or remove
  Groups as necessary to get your intended results structure.
  
except I don't add a pareAction ? Its probably easier to work out how to get what I need with v3.0 code.



In [14]:
import pyparsing as pp
vals = pp.OneOrMore(pp.pyparsing_common.integer)("int_values")

def add_total(tokens):
    tokens['total'] = sum(tokens)
    return tokens  # this line can be removed

vals.addParseAction(add_total)
print(vals.parseString("244 23 13 2343").dump())

[244, 23, 13, 2343]
- int_values: [244, 23, 13, 2343]
- total: 2623


In [15]:
result

ParseResults(['set', 'md', '50', ParseResults(['boty'], {'rule_name': ['boty'], 'rcall': [['boty']]}), ParseResults([ParseResults(['boty', ParseResults(['hue', '100.0'], {'tvalues': ['100.0']}), 'br', ParseResults([ParseResults(['rx', '180'], {'tid': 'rx', 'tvalues': ['180']}), ParseResults(['x', '20'], {'tid': 'x', 'tvalues': ['20']}), ParseResults(['rz', '2'], {'tid': 'rz', 'tvalues': ['2']})], {'trans': [{'tid': 'rx', 'tvalues': ['180']}, {'tid': 'x', 'tvalues': ['20']}, {'tid': 'rz', 'tvalues': ['2']}]}), 'br'], {'name': ['boty'], 'loop': {'trans': [{'tid': 'rx', 'tvalues': ['180']}, {'tid': 'x', 'tvalues': ['20']}, {'tid': 'rz', 'tvalues': ['2']}]}, 'rule_name': ['br'], 'rcall': [[{'tvalues': ['100.0']}, 'br'], [{'trans': [{'tid': 'rx', 'tvalues': ['180']}, {'tid': 'x', 'tvalues': ['20']}, {'tid': 'rz', 'tvalues': ['2']}]}, 'br']]}), ParseResults(['br', ParseResults([ParseResults(['rz', '60'], {'tid': 'rz', 'tvalues': ['60']})], {'trans': [{'tid': 'rz', 'tvalues': ['60']}]}), 'br3

In [16]:
print(result.dump())

['set', 'md', '50', ['boty'], [['boty', ['hue', '100.0'], 'br', [['rx', '180'], ['x', '20'], ['rz', '2']], 'br'], ['br', [['rz', '60']], 'br3', [['rz', '59']], 'br3', [['rz', '16'], ['s', '0.93'], ['x', '2']], 'br'], ['br3', [['x', '1'], ['y', '1'], ['s', '1']], 'sphere', [['x', '2'], ['y', '2']], 'box', [['rz', '1.6'], ['x', '1'], ['y', '1'], ['s', '0.980']], 'br3']]]
- entry_calls: [['boty']]
  [0]:
    ['boty']
    - rcall: [['boty']]
      [0]:
        ['boty']
    - rule_name: ['boty']
- global_md: '50'
- rule_defs: [[['boty', ['hue', '100.0'], 'br', [['rx', '180'], ['x', '20'], ['rz', '2']], 'br'], ['br', [['rz', '60']], 'br3', [['rz', '59']], 'br3', [['rz', '16'], ['s', '0.93'], ['x', '2']], 'br'], ['br3', [['x', '1'], ['y', '1'], ['s', '1']], 'sphere', [['x', '2'], ['y', '2']], 'box', [['rz', '1.6'], ['x', '1'], ['y', '1'], ['s', '0.980']], 'br3']]]
  [0]:
    [['boty', ['hue', '100.0'], 'br', [['rx', '180'], ['x', '20'], ['rz', '2']], 'br'], ['br', [['rz', '60']], 'br3', [['rz

In [17]:
# calculate using module
file_def =ex.eisen_grammar()
result = file_def.parseString(spiral)
xml = ex.es_xml2(result)
print('-----------')
print(ex.prettify(xml))

-----------
<?xml version="1.0" ?>
<rules max_depth="50">
  <rule name="entry">
    <call transforms="" rule=""/>
  </rule>
  <rule name="boty">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br3">
    <instance transforms="" shape=""/>
    <instance transforms="" shape=""/>
    <call transforms="" rule=""/>
  </rule>
</rules>



original
```
- rcall: 
  [0]:
    [['hue', '100.0'], 'br']
    - loop: 
      [0]:
        ['hue', '100.0']
        - tvalues: ['100.0']
    - rule_name: ['br']
```
now 
```
print(pcall.dump())
[['hue', '100.0'], 'br']
[0]:
  ['hue', '100.0']
  - tvalues: ['100.0']
[1]:
  br
```

In [18]:
# calculate using module
file_def =ex.eisen_grammar()
result = file_def.parseString(spiral)
xml = ex.es_xml2(result)
print('-----------')
print(ex.prettify(xml))

-----------
<?xml version="1.0" ?>
<rules max_depth="50">
  <rule name="entry">
    <call transforms="" rule=""/>
  </rule>
  <rule name="boty">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br">
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
    <call transforms="" rule=""/>
  </rule>
  <rule name="br3">
    <instance transforms="" shape=""/>
    <instance transforms="" shape=""/>
    <call transforms="" rule=""/>
  </rule>
</rules>



can I install an old version  
would it help?

I've done this in es_xml_pp2 and the code produces the correct results



In [19]:
print(file_def)

{['set' md | maxdepth W:((-+--9)] Group:({{{[Group:({[W:((-+--9) Suppress:('*')] Suppress:('{') [{Group:({x | y | z | s | rx | ry | rz {W:((-+--9)}...}) | {hue | h | sat | brightness | b | alpha | a | m {W:((-+--9)}...} | {'color' W:(#0-9A-Za-z)} | {'blend' W:(#0-9A-Za-z) W:((-+--9)}}]... Suppress:('}')})] Combine:({box | grid | sphere | line [W:(:A-Za-z)]})} | {[Group:({[W:((-+--9) Suppress:('*')] Suppress:('{') [{Group:({x | y | z | s | rx | ry | rz {W:((-+--9)}...}) | {hue | h | sat | brightness | b | alpha | a | m {W:((-+--9)}...} | {'color' W:(#0-9A-Za-z)} | {'blend' W:(#0-9A-Za-z) W:((-+--9)}}]... Suppress:('}')})]... {~{'rule'} W:(A-Za-z, 0-9A-Z_a-z)}}}}...) Group:({Group:({Suppress:('rule') {~{'rule'} W:(A-Za-z, 0-9A-Z_a-z)} {[md | maxdepth W:((-+--9) ['>' {~{'rule'} W:(A-Za-z, 0-9A-Z_a-z)}]] & [weight | w W:((-+--9)]} Suppress:('{') {{{[Group:({[W:((-+--9) Suppress:('*')] Suppress:('{') [{Group:({x | y | z | s | rx | ry | rz {W:((-+--9)}...}) | {hue | h | sat | brightness | b 

In [20]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""

In [21]:
result = file_def.parseString(ball)
xml = ex.es_xml2(result)
print('-----------')
print(ex.prettify(xml))

-----------
<?xml version="1.0" ?>
<rules max_depth="2000">
  <rule name="entry">
    <call transforms="" rule=""/>
  </rule>
  <rule name="R1" weight="10">
    <instance transforms="" shape=""/>
    <call transforms="" rule=""/>
  </rule>
</rules>



In [22]:
result

ParseResults(['set', 'maxdepth', '2000', ParseResults(['R1'], {'rule_name': ['R1'], 'rcall': [['R1']]}), ParseResults([ParseResults(['R1', 'w', '10', ParseResults([ParseResults(['x', '1'], {'tid': 'x', 'tvalues': ['1']}), ParseResults(['rz', '3'], {'tid': 'rz', 'tvalues': ['3']}), ParseResults(['ry', '5'], {'tid': 'ry', 'tvalues': ['5']})], {'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'rz', 'tvalues': ['3']}, {'tid': 'ry', 'tvalues': ['5']}]}), 'R1', ParseResults([ParseResults(['s', '1', '1', '0.1'], {'tid': 's', 'tvalues': ['1', '1', '0.1']}), 'sat', '0.9'], {'trans': [{'tid': 's', 'tvalues': ['1', '1', '0.1']}], 'tvalues': ['0.9']}), 'box'], {'name': ['R1'], 'wm': '10', 'loop': {'trans': [{'tid': 's', 'tvalues': ['1', '1', '0.1']}], 'tvalues': ['0.9']}, 'rule_name': ['R1'], 'rcall': [[{'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'rz', 'tvalues': ['3']}, {'tid': 'ry', 'tvalues': ['5']}]}, 'R1']], 'shape': 'box', 'bcall': [[{'trans': [{'tid': 's', 'tvalues': ['1', '1', '0.1

In [23]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""

trans_list gets something like 
`([(['s', '1', '1', '0.1'], {'tid': [('s', 0)], 'tvalues': [((['1', '1', '0.1'], {}), 1)]})], {})`
which it turns into  
`'s 1 1 0.1 '`






the needed stuff is still there multiple times

In [25]:
result.as_list()

['set',
 'maxdepth',
 '2000',
 ['R1'],
 [['R1',
   'w',
   '10',
   [['x', '1'], ['rz', '3'], ['ry', '5']],
   'R1',
   [['s', '1', '1', '0.1'], 'sat', '0.9'],
   'box']]]

In [26]:
result.as_dict()

{'global_md': '2000',
 'entry_calls': [{'rule_name': ['R1'], 'rcall': [['R1']]}],
 'rule_defs': [[{'name': ['R1'],
    'wm': '10',
    'loop': {'trans': [{'tid': 's', 'tvalues': ['1', '1', '0.1']}],
     'tvalues': ['0.9']},
    'rule_name': ['R1'],
    'rcall': [[{'trans': [{'tid': 'x', 'tvalues': ['1']},
        {'tid': 'rz', 'tvalues': ['3']},
        {'tid': 'ry', 'tvalues': ['5']}]},
      'R1']],
    'shape': 'box',
    'bcall': [[{'trans': [{'tid': 's', 'tvalues': ['1', '1', '0.1']}],
       'tvalues': ['0.9']},
      'box']]}]]}

In [27]:
result['global_md']

'2000'

In [28]:
result.global_md

'2000'

In [31]:
for p in result.entry_calls:
    print(p.rule_name[0])

    

R1


In [33]:
len(p.loop)

0

In [34]:
p.trans

''